In [2]:
import scipy as scp
import numpy as np



print("hello world")


hello world


In [13]:
def initialize(inPos,inVel,stepsize=0.1):
    G0 = 2
    d = 1
    M_e = 1
    M_m = 1

    
    global pos
    global vel
    pos = np.array(inPos)
    vel = np.array(inVel)
initialize([1,2,3],[4,5,6])
print(pos)
print(vel)

[1 2 3]
[4 5 6]
